In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices()
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[1], True)

2022-05-18 22:02:16.761144: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-05-18 22:02:19.097648: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-18 22:02:19.101659: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-18 22:02:19.163763: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 22:02:19.164379: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 119.24GiB/s
2022-05-18 22:02:19.164440: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-05-18 22:02:19.191699: I tensorflow/stream_executor/platform/def

In [2]:
import numpy as np
import math
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from collections import deque
import gym
import os
from tensorflow.keras.optimizers import Adam
import copy

In [3]:
env = gym.make("BipedalWalker-v3")
input_size = env.observation_space.shape[0]
output_size = env.action_space.shape[0]
input_size, output_size

(24, 4)

In [4]:
env = gym.wrappers.RecordVideo(env, "videos", step_trigger=lambda t: t%100==0)

/home/himanshu/anaconda3/envs/myenv/lib/python3.9/site-packages/gym/wrappers/record_video.py:41: UserWarning: WARN: Overwriting existing videos at /home/himanshu/Jupyter/Jupyter Notebook/DDPG Trials/Bipedal/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [5]:
tau = 0.001             #soft target update
gamma = 0.99             #discount factor


actor_lr = 0.0001       
critic_lr = 0.001

memory_size = int(1e6)
minibatch_size = 128

num_episodes = 2_000   #max episodes for training
num_steps=700          #max number of steps in an episode

noise_std = 0.2        #stddev of noise added for exploration
start_after = 1000     #pure exploration before 1000 steps
update_after = 1000    #start learning after 1000 steps

l2_decay=0.001         #weight decay for critic network

In [6]:
import datetime
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# rm -rf ./logs/train_PPO/
train_log_dir = "logs/train_DDPG/" + current_time
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
%load_ext tensorboard
%tensorboard --logdir {train_log_dir}

2022-05-18 22:02:19.544433: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-18 22:02:19.545038: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-18 22:02:19.545246: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 16 deviceMemorySize: 3.81GiB deviceMemoryBandwidth: 119.24GiB/s
2022-05-18 22:02:19.545278: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libc

In [7]:
class ReplayMemory:
    def __init__(self, input_size, output_size, size):
        self.states = np.zeros(shape = (size, input_size), dtype=np.float32)
        self.next_states = np.zeros(shape = (size, input_size), dtype=np.float32)
        self.actions = np.zeros(shape = (size, output_size), dtype=np.float32)
        self.rewards = np.zeros(shape = size, dtype=np.float32)
        self.dones = np.zeros(shape = size, dtype=np.float32)
        self.pointer=0
        self.size=0
        self.max_size=size
    
    def remember(self, state, action, reward, next_state, done):
        self.states[self.pointer] = state
        self.next_states[self.pointer] = next_state
        self.actions[self.pointer] = action
        self.rewards[self.pointer] =  reward
        self.dones[self.pointer] = done
        self.pointer = (self.pointer + 1)%self.max_size
        self.size = min(self.size+1, self.max_size)
        
    def sample(self, minibatch_size = 32):
        idxs = np.random.randint(0, self.size, size=minibatch_size)
        
        return self.states[idxs],\
               self.actions[idxs],\
               self.rewards[idxs],\
               self.next_states[idxs],\
               self.dones[idxs]
                
    

In [8]:
class Agent:
    def __init__(self):
        self.actor = self.build_actor()
        self.critic = self.build_critic()
        
        self.target_actor = self.build_actor()
        self.target_critic = self.build_critic()
        
        self.target_actor.set_weights(self.actor.get_weights())
        self.target_critic.set_weights(self.critic.get_weights())
        
        self.actor_optimizer = Adam(lr=actor_lr)
        self.critic_optimizer =  Adam(lr=critic_lr)
        
        self.memory = ReplayMemory(input_size, output_size, memory_size)
#         self.noise = NoiseProcess()
        
    def build_actor(self):
        init = tf.random_uniform_initializer(minval=-3e-3, maxval=3e-3)
        
        
        inputs = Input(shape = input_size)
        x = Dense(400, activation = 'relu')(inputs)
        x = Dense(300, activation = 'relu')(x)
        x= Dense(100, activation="relu")(x)
        x = Dense(output_size, activation = "tanh", kernel_initializer=init)(x)
        model = Model(inputs = inputs, outputs = x)
        return model
    
    def build_critic(self):
        init = tf.random_uniform_initializer(minval=-3e-3, maxval=3e-3)
        regularizer = tf.keras.regularizers.L2(l2=l2_decay)
        
        input1 = Input(shape = input_size)
        input2 = Input(shape = output_size)
        x = Dense(400, activation="relu", kernel_regularizer=regularizer)(input1)
        x = Dense(300, activation="relu", kernel_regularizer=regularizer)(tf.concat([x, input2], axis =1))
        x = Dense(100, activation="relu", kernel_regularizer=regularizer)(x)
        x = Dense(1, activation = 'linear', kernel_initializer=init, kernel_regularizer=regularizer)(x)
        model = Model(inputs = [input1, input2], outputs = x)
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))
        
    @tf.function 
    def act(self, state, test = False, noise_std=0.2):
        act_value = self.actor(state)
        if not test:
            act_value += tf.random.normal(shape = act_value.shape, mean=0.0, stddev=noise_std)
        return tf.clip_by_value(act_value, -1, 1)
    
    
    @tf.function
    def optimize(self, states, actions, rewards, next_states, dones):
        rewards= tf.expand_dims(rewards, axis=1)
        dones = tf.expand_dims(dones, axis=1)
        next_actions = self.target_actor(next_states)
        next_values = self.target_critic([next_states, next_actions])
        target_q_values = rewards + gamma * (1-dones) * next_values
        with tf.GradientTape() as tape:        
            pred_q_values = self.critic([states, actions])
            critic_loss = tf.reduce_mean((target_q_values - pred_q_values)**2)
        critic_grads = tape.gradient(critic_loss, self.critic.trainable_weights)
        self.critic_optimizer.apply_gradients(zip(critic_grads, self.critic.trainable_weights))
        
        with tf.GradientTape() as tape:
            actions_pred = self.actor(states)
            actor_loss = -self.critic([states, actions_pred])
            actor_loss = tf.reduce_mean(actor_loss)
        actor_grads = tape.gradient(actor_loss, self.actor.trainable_weights)
        self.actor_optimizer.apply_gradients(zip(actor_grads, self.actor.trainable_weights))
    
    def learn(self):
        if agent.memory.size<minibatch_size:
            return 
        states, actions, rewards, next_states, dones = self.memory.sample(minibatch_size)
        self.optimize(states, actions, rewards, next_states, dones) 
        self.update_target_weights()
        
    def update_target_weights(self):
        target_actor_weights = self.target_actor.get_weights()
        weights=[]
        for i, weight in enumerate(self.actor.get_weights()):
            weights.append(weight * tau + target_actor_weights[i] * (1-tau))
        self.target_actor.set_weights(weights)
        
                           
        target_critic_weights = self.target_critic.get_weights()
        weights=[]
        for i, weight in enumerate(self.critic.get_weights()):
            weights.append(weight * tau + target_critic_weights[i] * (1-tau))
        self.target_critic.set_weights(weights)
                           


    def load(self, name):
        self.actor.load_weights(name + "actor.hdf5")
        self.critic.load_weights(name + "critic.hdf5")
        self.target_actor.load_weights(name + "actor.hdf5")
        self.target_critic.load_weights(name + "critic.hdf5")
    
    def save(self, name):
        self.actor.save_weights(name + "actor.hdf5")
        self.critic.save_weights(name + "critic.hdf5")    

In [9]:
agent = Agent()

In [ ]:
#Training 
best_score = env.reward_range[0]
score_history = deque(maxlen=100)
global_step = 0
for ep in range(0, num_episodes):
    state = env.reset()
    done = False
    score = 0
    for step in range(num_steps):
        if global_step<start_after:
            action = env.action_space.sample()
        else:
            action =  agent.act(np.expand_dims(state, axis = 0), noise_std = noise_std)
            action = np.squeeze(action)
            
        next_state, reward, done, _ =  env.step(action)
        score += reward
        global_step += 1
        agent.memory.remember(state, action, reward, next_state, done)
        state = next_state
        if global_step>update_after:
            agent.learn()
            
        if done:
            break
    with train_summary_writer.as_default():
        tf.summary.scalar("Charts/score", score, global_step)
        tf.summary.scalar("Charts/episode_length", step, global_step)
        tf.summary.scalar("Charts/exploration", noise_std, global_step)
        
    score_history.append(score)
    avg_score = np.mean(score_history)
    print(f"Episode: {ep}, Len: {step}, Score: {score}, Avg Score: {avg_score}, Global_Step: {global_step}")  
    if avg_score > best_score:
        best_score = score
        agent.save("DDPG_") 
    if ep%25==0:
        print(f"Testing")
        for _ in range(2):
            state=env.reset()
            done = False
            test_score=0
            test_step=0
            while not done:
                action = agent.act(np.expand_dims(state, axis =0), test=True)
                next_state, reward, done, _ = env.step(np.squeeze(action))
                test_score+= reward
                test_step+=1
                state=next_state
            print(f"Episode ended with length {test_step} and a score of {test_score}")


In [10]:
#Loading pretrained weights

agent.load("DDPG_")

In [11]:
#Testing
done = False
state = env.reset()
score=0
step = 0
while not done:
    env.render()
    action = agent.act(np.expand_dims(state, axis=0), test=True)
    next_state, reward, done, _ = env.step(np.squeeze(action))
    score+= reward
    state = next_state
    print(action, reward)
    step += 1
print(f"Episode ended in {step} steps with score of {score}")
env.close()

2022-05-18 22:02:26.770975: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-18 22:02:26.790690: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400000000 Hz
2022-05-18 22:02:26.817353: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


tf.Tensor([[-0.99376744 -0.99999976 -0.99999833  0.99999976]], shape=(1, 4), dtype=float32) -0.2082012184392661
tf.Tensor([[ 0.9999569   0.9999834  -0.99999976 -0.9520176 ]], shape=(1, 4), dtype=float32) -0.20768688043951988
tf.Tensor([[-0.99999934 -0.99999976  0.9693347   0.999996  ]], shape=(1, 4), dtype=float32) -0.1939629460076503
tf.Tensor([[-0.99997866  0.7782905  -0.99999976  0.938457  ]], shape=(1, 4), dtype=float32) -0.22653241705894472
tf.Tensor([[-0.99741447 -0.99980825 -0.38631767  0.99999976]], shape=(1, 4), dtype=float32) -0.3320504179795571
tf.Tensor([[-0.9998638   0.99999976 -0.99992794  0.99999964]], shape=(1, 4), dtype=float32) -0.3507051118214913
tf.Tensor([[-0.99915075  0.9290743   0.96210456  0.99999976]], shape=(1, 4), dtype=float32) -0.2791306826273612
tf.Tensor([[-0.98559344 -0.9997092  -0.93184876  0.99999976]], shape=(1, 4), dtype=float32) -0.28147970823447027
tf.Tensor([[ 0.99984777  0.99999976 -0.99999976  0.999997  ]], shape=(1, 4), dtype=float32) -0.244527

tf.Tensor([[-0.9993979  -0.99999976  0.99975854  0.99999976]], shape=(1, 4), dtype=float32) 0.2523677537441254
tf.Tensor([[ 0.5137975  -0.9950612  -0.89164984  0.95694065]], shape=(1, 4), dtype=float32) 0.23982454967498776
tf.Tensor([[-0.8234788  -0.9944647  -0.99767834  0.90977645]], shape=(1, 4), dtype=float32) 0.17926866396268448
tf.Tensor([[ 0.9837533   0.95458746 -0.99995846 -0.99999815]], shape=(1, 4), dtype=float32) 0.17251586031913757
tf.Tensor([[-0.99994904  0.9346474  -0.8543012  -0.99942756]], shape=(1, 4), dtype=float32) 0.09459037184715273
tf.Tensor([[-0.02489552 -0.6952867   0.24281919 -0.9999989 ]], shape=(1, 4), dtype=float32) 0.1641839362780229
tf.Tensor([[ 0.968297   -0.73338765  0.86099637 -0.9999997 ]], shape=(1, 4), dtype=float32) 0.258678916573528
tf.Tensor([[-0.9999779  0.6676785 -0.646507  -0.9999969]], shape=(1, 4), dtype=float32) 0.13383753752708433
tf.Tensor([[-0.18343034  0.9972559   0.8998617  -0.99999976]], shape=(1, 4), dtype=float32) 0.16249096457163253


tf.Tensor([[-0.7844377  -0.8322385   0.98656803 -0.99524224]], shape=(1, 4), dtype=float32) 0.17653353234131655
tf.Tensor([[-0.65777326  0.6680201   0.9917429  -0.99986726]], shape=(1, 4), dtype=float32) 0.24676548214753707
tf.Tensor([[-0.99304634  0.9999351   0.999459   -0.99997485]], shape=(1, 4), dtype=float32) 0.177200325806938
tf.Tensor([[-0.9428062   0.98949635  0.9855605  -0.99779713]], shape=(1, 4), dtype=float32) 0.11413451365629596
tf.Tensor([[ 0.998814   -0.98797876 -0.6290724   0.9847575 ]], shape=(1, 4), dtype=float32) 0.15679729251066607
tf.Tensor([[ 0.9998956  -0.9998062  -0.99971235  0.9974875 ]], shape=(1, 4), dtype=float32) 0.14911533220609266
tf.Tensor([[ 0.9999592  -0.99999595 -0.5675836   0.5402124 ]], shape=(1, 4), dtype=float32) 0.1627716052929537
tf.Tensor([[ 0.997955   -0.99999267  0.96149164  0.95110494]], shape=(1, 4), dtype=float32) 0.1973368254502638
tf.Tensor([[ 0.94826144 -0.99999976  0.81761     0.99999976]], shape=(1, 4), dtype=float32) 0.25638052217165

tf.Tensor([[ 0.9989498 -0.9999324  0.7965608  0.9993223]], shape=(1, 4), dtype=float32) 0.21150920327504003
tf.Tensor([[ 0.02158879 -0.9998863   0.92219     0.8351955 ]], shape=(1, 4), dtype=float32) 0.3097934934919174
tf.Tensor([[-0.99959254 -0.99999976  0.90531164  0.9999675 ]], shape=(1, 4), dtype=float32) 0.27015046779313734
tf.Tensor([[-0.99854064 -0.26577613 -0.9784678  -0.96187   ]], shape=(1, 4), dtype=float32) 0.1665704979499276
tf.Tensor([[-0.83270377 -0.5958701  -0.99602467 -0.35629302]], shape=(1, 4), dtype=float32) 0.1737810440460746
tf.Tensor([[ 0.34488302  0.9837934   0.21298584 -0.99988437]], shape=(1, 4), dtype=float32) 0.2215648606220928
tf.Tensor([[ 0.8545471   0.59737605  0.9987502  -0.99999976]], shape=(1, 4), dtype=float32) 0.3380686384042057
tf.Tensor([[-0.9960243   0.72275954  0.9993251  -0.99999917]], shape=(1, 4), dtype=float32) 0.304486637274429
tf.Tensor([[-0.9993416   0.91263115  0.99996585 -0.99997747]], shape=(1, 4), dtype=float32) 0.21003089598815117
tf.

...

tf.Tensor([[ 0.85899717 -0.99999523  0.99901396  0.94151527]], shape=(1, 4), dtype=float32) 0.27366899589693733
tf.Tensor([[-0.8272702  -0.99999976  0.9967018   0.99999934]], shape=(1, 4), dtype=float32) 0.28542258389794645
tf.Tensor([[ 0.93166727 -0.99785876 -0.98967344  0.85966575]], shape=(1, 4), dtype=float32) 0.28042026575404505
tf.Tensor([[-0.7186145  -0.99997276 -0.98439866  0.39227456]], shape=(1, 4), dtype=float32) 0.2108824156522751
tf.Tensor([[-0.6688944 -0.9756749 -0.9999497 -0.9884395]], shape=(1, 4), dtype=float32) 0.09160416583222057
tf.Tensor([[-0.98081315  0.92672455 -0.9966065  -0.9985756 ]], shape=(1, 4), dtype=float32) 0.09752614267665238
tf.Tensor([[-0.47387034  0.9922776  -0.8450381  -0.99999964]], shape=(1, 4), dtype=float32) 0.11717906788986832
tf.Tensor([[ 0.10611734  0.9257465   0.9137502  -0.99999976]], shape=(1, 4), dtype=float32) 0.19732519638538365
tf.Tensor([[-0.77939224  0.8879347   0.9169274  -0.99999976]], shape=(1, 4), dtype=float32) 0.174184925953509

tf.Tensor([[ 0.01603419 -0.99999964  0.99999547 -0.23769537]], shape=(1, 4), dtype=float32) 0.33897691578418016
tf.Tensor([[-0.44236952 -0.99999976  0.99999976  0.99975556]], shape=(1, 4), dtype=float32) 0.2985984332959047
tf.Tensor([[-0.9822231  -0.99999976  0.99791247  0.99999976]], shape=(1, 4), dtype=float32) 0.2831421251296997
tf.Tensor([[ 0.9707443  -0.9991293  -0.99883956  0.8516355 ]], shape=(1, 4), dtype=float32) 0.2859597313404083
tf.Tensor([[ 0.81731147 -0.99999976 -0.99860257  0.99999976]], shape=(1, 4), dtype=float32) 0.2390505054791579
tf.Tensor([[ 0.23037641 -0.99998504 -0.99304324  0.99999046]], shape=(1, 4), dtype=float32) 0.2647098029255867
tf.Tensor([[-0.5532023  -0.9999214  -0.99741524  0.9726224 ]], shape=(1, 4), dtype=float32) 0.28574817345541625
tf.Tensor([[ 0.7624291   0.31890246 -0.9999933  -0.9965167 ]], shape=(1, 4), dtype=float32) 0.28144208025932305
tf.Tensor([[-0.99999976  0.96768314 -0.9725268  -0.99906707]], shape=(1, 4), dtype=float32) 0.202719864288928

tf.Tensor([[ 0.98907363 -0.98124343 -0.91186553  0.94470584]], shape=(1, 4), dtype=float32) 0.2819274568359247
tf.Tensor([[ 0.99999404 -0.9999975  -0.99891853  0.9999996 ]], shape=(1, 4), dtype=float32) 0.29278485300141666
tf.Tensor([[ 0.99995744 -0.99999976 -0.99994916  0.99999976]], shape=(1, 4), dtype=float32) 0.25810428160429005
tf.Tensor([[ 0.9863346  -0.99999976 -0.9778589   0.99999976]], shape=(1, 4), dtype=float32) 0.2637521906296602
tf.Tensor([[-0.8770138  -0.99999976  0.16657959  0.99999976]], shape=(1, 4), dtype=float32) 0.22931604935723637
tf.Tensor([[-0.6918414  -0.9999997  -0.36078167  0.99999976]], shape=(1, 4), dtype=float32) 0.19828697697323472
tf.Tensor([[ 0.69791573 -0.9992952  -0.99492306  0.3103756 ]], shape=(1, 4), dtype=float32) 0.30685801553726194
tf.Tensor([[-0.9991926 -0.9996101 -0.7147519  0.9574529]], shape=(1, 4), dtype=float32) 0.26000617893534994
tf.Tensor([[-0.98722345  0.696447   -0.93570524 -0.99999976]], shape=(1, 4), dtype=float32) 0.2432935298283705

tf.Tensor([[ 0.99989766 -0.99999976  0.97209436  0.9999079 ]], shape=(1, 4), dtype=float32) 0.38087358681362776
tf.Tensor([[-0.33228278  0.9647197  -0.2685443  -0.9982486 ]], shape=(1, 4), dtype=float32) 0.3235807563066483
tf.Tensor([[-0.9941756   0.9958135   0.7887504  -0.99546987]], shape=(1, 4), dtype=float32) 0.257459163983644
tf.Tensor([[ 0.6864902  -0.6279062   0.01462662 -0.7254594 ]], shape=(1, 4), dtype=float32) 0.36229070867353497
tf.Tensor([[ 0.9999116  -0.99997085  0.69702333  0.98443466]], shape=(1, 4), dtype=float32) 0.3557597826719284
tf.Tensor([[ 0.9987438  -0.9999443  -0.69535154  0.8365114 ]], shape=(1, 4), dtype=float32) 0.32340300822257995
tf.Tensor([[ 0.99684095 -0.99999976 -0.99959564  0.9999991 ]], shape=(1, 4), dtype=float32) 0.2632289553880691
tf.Tensor([[ 0.998413   -0.99999976 -0.9921555   0.99999976]], shape=(1, 4), dtype=float32) 0.2146263917684555
tf.Tensor([[ 0.9931975  -0.99999976  0.9002933   0.99999976]], shape=(1, 4), dtype=float32) 0.2734331845839629

tf.Tensor([[-0.21666011 -0.99999946 -0.88356197 -0.5083955 ]], shape=(1, 4), dtype=float32) 0.293134374499321
tf.Tensor([[-0.9154695  -0.9999202  -0.27039275 -0.27413172]], shape=(1, 4), dtype=float32) 0.2562273151477557
tf.Tensor([[ 0.3924884  -0.9920873   0.44299173 -0.9561706 ]], shape=(1, 4), dtype=float32) 0.31397063616907783
tf.Tensor([[-0.51580614 -0.63432693  0.97737163 -0.99591696]], shape=(1, 4), dtype=float32) 0.33585638046264654
tf.Tensor([[-0.9998862   0.99231774  0.99999106 -0.99995077]], shape=(1, 4), dtype=float32) 0.3508881384531846
tf.Tensor([[-0.9819344  0.8207383  0.9999395 -0.9953587]], shape=(1, 4), dtype=float32) 0.37161009339493417
tf.Tensor([[-0.7227946   0.9767365   0.98525816 -0.9922106 ]], shape=(1, 4), dtype=float32) 0.375868778149249
tf.Tensor([[-0.99982697  0.9999989   0.9996294  -0.9999966 ]], shape=(1, 4), dtype=float32) 0.28827467338246016
tf.Tensor([[-0.9263419   0.7000319   0.42553118 -0.8528736 ]], shape=(1, 4), dtype=float32) 0.253569316069304
tf.T